In [4]:
# !pip install "flaml[automl]"
# !pip install rapidfuzz

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os

In [ ]:
from nq_agents import WorkflowAutogen
from nq_agents import data_extraction

# Initialize the multi-agent system
my_agents = WorkflowAutogen()

# Read input examples
GOLD_FILE = 'data/v1.0-simplified_nq-dev-all_sample40_seed32.jsonl'
context_path = GOLD_FILE.replace('.jsonl', '_context.jsonl')
save_path = GOLD_FILE.replace('.jsonl', '_predictions.json')

# Count the number of lines in the file
num_examples = data_extraction.count_lines(GOLD_FILE)

input_examples = data_extraction.read_first_lines(GOLD_FILE, num_examples)

# Make predictions. side effect: write to file
input_examples = input_examples[:]
my_agents.predict_batch(examples=input_examples, context_path=context_path, verbose=True)

predictions = my_agents.read_log_and_format(context_path=context_path, save_path=save_path)

print("Two files are generated: \n", context_path, "\n", save_path)
print("Make sure to rename the files unless you want to append to the end.")


### Evaluate


In [ ]:
from nq_agents import evaluation

# Evaluate predictions
PREDICTIONS_FILE = save_path # generated by multi_agent.py

result = evaluation.evaluate_predictions(GOLD_FILE, PREDICTIONS_FILE)

### Evaluation by text

In [11]:
context_path = 'data/llm+rank+refine+instruct+40_context.jsonl'


In [ ]:
import json
from nq_agents.multi_agent import get_short_answers

correct_count = 0
total_count = 0
with open(context_path, 'r') as f:
    for line in f:
        context = json.loads(line)
        cut_answer = context['cut_answer']
        true_answers = get_short_answers(context['example'])
        if cut_answer in true_answers:
            correct_count += 1
        total_count += 1

print(f"Accuracy: {correct_count / total_count}")

